## 🔹 Estratégia de Extração: Segmentos de 10 segundos

Nesta primeira etapa, optamos por extrair características diretamente de **segmentos de 10 segundos** dos sinais de fonocardiograma (FCG), sem dividi-los em ciclos cardíacos individuais.

### 🎯 Justificativa

Todos os sinais disponíveis no dataset já estão organizados em arquivos com **duração fixa de 10 segundos**, o que oferece uma base consistente para a extração de características. A escolha por manter essa estrutura se baseia em:

- **Simplicidade de implementação:** evita a complexidade adicional da segmentação por batimentos cardíacos.
- **Padronização dos dados:** todos os exemplos possuem o mesmo comprimento temporal e amostral (10 s a 2000 Hz = 20.000 amostras).
- **Captação de padrões médios:** os sinais de FCG, mesmo em 10 segundos, geralmente contêm múltiplos ciclos cardíacos (~10 a 20), o que permite capturar tendências estatísticas globais do som.
- **Redução de ruído:** ao considerar um período maior, minimizamos variações batimento-a-batimento que podem ser irrelevantes ou ruidosas para o modelo.

### 🔍 Considerações futuras

Embora a segmentação por ciclos cardíacos (ex: detecção de S1 e S2) possa oferecer análises mais detalhadas, esta abordagem será considerada como uma possível **etapa posterior**, após a validação de um pipeline funcional baseado nos segmentos completos.

Esta decisão permite focar inicialmente na **construção de um modelo robusto**, aproveitando a estrutura existente do dataset.



In [3]:
# Notebook: 01_leitura_fcgs.ipynb

import os
import librosa
import pandas as pd
from tqdm import tqdm

# Caminho base dos dados
base_path = "../Database/Bandpass_signal/78_participants_BPF_7_segment"

# Lista de participantes
participants = [f"Participant_BPF_{str(i).zfill(2)}" for i in range(1, 79)]

# Lista para armazenar metadados dos arquivos
file_metadata = []

# Leitura dos arquivos
for participant in tqdm(participants):
    participant_path = os.path.join(base_path, participant)
    
    if not os.path.isdir(participant_path):
        continue

    for seg_num in range(3, 10):
        file_name = f"segment_{seg_num}.wav"
        file_path = os.path.join(participant_path, file_name)

        if os.path.exists(file_path):
            # Tenta carregar o áudio para verificar integridade
            try:
                y, sr = librosa.load(file_path, sr=None)
                file_metadata.append({
                    "participant": participant,
                    "segment": seg_num,
                    "file_path": file_path,
                })
            except Exception as e:
                print(f"Erro ao carregar {file_path}: {e}")

# Converte em DataFrame
df_files = pd.DataFrame(file_metadata)

# Visualiza as 5 primeiras linhas
df_files.head()


100%|██████████| 78/78 [00:00<00:00, 220.80it/s]


,participant,segment,file_path
0,Participant_BPF_01,3,../Database/Bandpass_signal/78_participants_BP...
1,Participant_BPF_01,4,../Database/Bandpass_signal/78_participants_BP...
2,Participant_BPF_01,5,../Database/Bandpass_signal/78_participants_BP...
3,Participant_BPF_01,6,../Database/Bandpass_signal/78_participants_BP...
4,Participant_BPF_01,7,../Database/Bandpass_signal/78_participants_BP...


## 🕒 Características no Domínio do Tempo

| Nome               | Descrição |
|--------------------|-----------|
| **RMS (Root Mean Square)** | Representa a energia média do sinal. Calcula-se como a média quadrática da amplitude. Sons cardíacos com alterações patológicas podem apresentar mudanças na energia do sinal. |
| **ZCR (Zero Crossing Rate)** | Mede a taxa média de cruzamentos por zero. É uma estimativa da frequência ou do conteúdo de ruído de um sinal. Sons cardíacos mais ruidosos (ex: sopros) tendem a ter ZCR mais alto. |
| **Amplitude Range** | Diferença entre a maior e a menor amplitude do sinal. Indica a extensão dinâmica do batimento. Pode ser útil para detectar batimentos com picos anormais. |
| **Max Amplitude** | Valor máximo da amplitude do sinal. Um valor muito alto pode indicar ruído ou batimentos intensos. |
| **Min Amplitude** | Valor mínimo da amplitude do sinal. Assim como o máximo, ajuda a descrever a dinâmica geral do som. |
| **Skewness (Assimetria)** | Mede a assimetria da distribuição das amplitudes. Valores positivos ou negativos indicam inclinação do sinal para direita ou esquerda, o que pode ocorrer em ciclos cardíacos anormais. |
| **Kurtosis (Curtose)** | Mede o quão "pontuda" ou achatada é a distribuição do sinal. Altos valores indicam presença de picos ou eventos súbitos, como estalidos. |


In [5]:
import pandas as pd
import numpy as np
import librosa
from scipy.stats import skew, kurtosis
from tqdm import tqdm


# Lista para armazenar as features
time_features = []

# Extrai características do tempo
for idx, row in tqdm(df_files.iterrows(), total=len(df_files)):
    file_path = row["file_path"]
    
    try:
        y, sr = librosa.load(file_path, sr=None)
        
        # Características no tempo
        rms = np.mean(librosa.feature.rms(y=y))
        zcr = np.mean(librosa.feature.zero_crossing_rate(y))
        amp_range = np.max(y) - np.min(y)
        max_amp = np.max(y)
        min_amp = np.min(y)
        skewness = skew(y)
        kurt = kurtosis(y)

        time_features.append({
            "participant": row["participant"],
            "segment": row["segment"],
            "rms": rms,
            "zcr": zcr,
            "amplitude_range": amp_range,
            "max_amplitude": max_amp,
            "min_amplitude": min_amp,
            "skewness": skewness,
            "kurtosis": kurt
        })
    
    except Exception as e:
        print(f"Erro ao processar {file_path}: {e}")

# Cria DataFrame com as features
df_time = pd.DataFrame(time_features)

# Salva para uso futuro
df_time.to_csv("fcg_features_tempo.csv", index=False)
df_time.head()


100%|██████████| 546/546 [00:06<00:00, 90.71it/s] 


,participant,segment,rms,zcr,amplitude_range,max_amplitude,min_amplitude,skewness,kurtosis
0,Participant_BPF_01,3,0.005612,0.091980,0.226166,0.096802,-0.129364,-1.261252,85.235977
1,Participant_BPF_01,4,0.004565,0.091516,0.226166,0.096802,-0.129364,-1.157882,101.555847
2,Participant_BPF_01,5,0.003261,0.089685,0.211761,0.104584,-0.107178,-0.070009,203.891434
3,Participant_BPF_01,6,0.003473,0.091467,0.211761,0.104584,-0.107178,-0.208206,202.283752
4,Participant_BPF_01,7,0.002359,0.091577,0.088806,0.046875,-0.041931,-0.640040,48.104519


## Características Extraídas no Domínio da Frequência

| Característica             | Descrição |
|----------------------------|-----------|
| **Frequência Média (Mean Frequency)** | Média ponderada das frequências presentes no espectro, considerando as amplitudes como pesos. Indica onde está concentrada a energia do sinal. |
| **Frequência Mediana (Median Frequency)** | Frequência que divide o espectro em duas partes com a mesma energia total. Pode indicar alterações fisiológicas. |
| **Frequência de Pico (Peak Frequency)** | Frequência com maior magnitude no espectro. Representa o componente mais dominante do sinal. |
| **Energia Espectral (Spectral Energy)** | Soma das energias de todas as componentes espectrais. Mede a potência total do sinal no domínio da frequência. |
| **Largura de Banda (Bandwidth)** | Medida da dispersão do espectro em torno da frequência central. Indica o quanto o sinal é concentrado ou espalhado. |
| **Skewness Espectral** | Grau de assimetria da distribuição espectral. Pode sugerir deslocamento da energia para frequências mais altas ou mais baixas. |
| **Curtose Espectral** | Mede a "picozação" do espectro, ou seja, quão concentrada está a energia em torno da média. Alta curtose indica um espectro mais afilado. |


In [6]:
import pandas as pd
import numpy as np
import librosa
from scipy.stats import skew, kurtosis
from tqdm import tqdm

# Lista para armazenar as features de frequência
freq_features = []

# Extrai características no domínio da frequência
for idx, row in tqdm(df_files.iterrows(), total=len(df_files)):
    file_path = row["file_path"]
    
    try:
        y, sr = librosa.load(file_path, sr=None)
        
        # Transformada de Fourier (espectro)
        spectrum = np.abs(np.fft.fft(y))
        freqs = np.fft.fftfreq(len(y), d=1/sr)
        
        # Usa apenas parte positiva do espectro
        pos_mask = freqs >= 0
        freqs = freqs[pos_mask]
        spectrum = spectrum[pos_mask]
        
        # Características espectrais
        spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        spectral_flatness = np.mean(librosa.feature.spectral_flatness(y=y))
        
        # Estatísticas do espectro
        spec_skewness = skew(spectrum)
        spec_kurtosis = kurtosis(spectrum)

        freq_features.append({
            "participant": row["participant"],
            "segment": row["segment"],
            "spectral_centroid": spectral_centroid,
            "spectral_bandwidth": spectral_bandwidth,
            "spectral_rolloff": spectral_rolloff,
            "spectral_flatness": spectral_flatness,
            "spectrum_skewness": spec_skewness,
            "spectrum_kurtosis": spec_kurtosis
        })
    
    except Exception as e:
        print(f"Erro ao processar {file_path}: {e}")

# Cria DataFrame com as features
df_freq = pd.DataFrame(freq_features)

# Salva para uso futuro
df_freq.to_csv("fcg_features_frequencia.csv", index=False)
df_freq.head()


100%|██████████| 546/546 [00:05<00:00, 98.72it/s] 


,participant,segment,spectral_centroid,spectral_bandwidth,spectral_rolloff,spectral_flatness,spectrum_skewness,spectrum_kurtosis
0,Participant_BPF_01,3,140.575678,86.799827,216.552734,0.000931,2.988137,10.311388
1,Participant_BPF_01,4,135.789629,86.412846,207.666016,0.001128,3.030209,10.126452
2,Participant_BPF_01,5,133.098592,85.853982,206.103516,0.000918,2.987428,9.797406
3,Participant_BPF_01,6,140.257409,93.445286,217.333984,0.002527,2.968128,10.411150
4,Participant_BPF_01,7,139.385806,93.163310,218.066406,0.002332,3.234632,13.272388


### Características no Domínio Cepstral (MFCCs)

As **Mel-Frequency Cepstral Coefficients (MFCCs)** são amplamente utilizadas para representar sinais de áudio, especialmente em tarefas de reconhecimento de fala e classificação de sons. Elas representam a envoltória espectral do som em uma escala perceptual (escala Mel), aproximando a forma como o ouvido humano percebe frequências.

As seguintes características são extraídas para cada um dos 13 primeiros coeficientes MFCC:

| Nome da Feature          | Descrição                                                                 |
|--------------------------|---------------------------------------------------------------------------|
| `mfcc_i_mean`            | Média do i-ésimo coeficiente MFCC                                         |
| `mfcc_i_std`             | Desvio padrão do i-ésimo coeficiente MFCC                                 |
| `mfcc_i_skew`            | Assimetria (skewness) da distribuição do i-ésimo MFCC                      |
| `mfcc_i_kurtosis`        | Curtose da distribuição do i-ésimo MFCC                                    |

Além dos MFCCs originais, também são computadas suas derivadas de primeira e segunda ordem:

| Nome da Feature              | Descrição                                                                 |
|------------------------------|---------------------------------------------------------------------------|
| `delta_i_mean`               | Média da derivada de 1ª ordem (delta) do i-ésimo MFCC                     |
| `delta_i_std`                | Desvio padrão da derivada de 1ª ordem                                     |
| `delta_i_skew`               | Assimetria da derivada de 1ª ordem                                        |
| `delta_i_kurtosis`           | Curtose da derivada de 1ª ordem                                           |
| `delta2_i_mean`              | Média da derivada de 2ª ordem (delta-delta) do i-ésimo MFCC               |
| `delta2_i_std`               | Desvio padrão da derivada de 2ª ordem                                     |
| `delta2_i_skew`              | Assimetria da derivada de 2ª ordem                                        |
| `delta2_i_kurtosis`          | Curtose da derivada de 2ª ordem                                           |

> **Nota:** As estatísticas são calculadas para cada um dos 13 coeficientes MFCC (`i = 1..13`). Isso fornece uma representação robusta das características cepstrais e suas variações ao longo do tempo.


In [7]:
import pandas as pd
import numpy as np
import librosa
from scipy.stats import skew, kurtosis
from tqdm import tqdm

# Lista para armazenar as features MFCC
mfcc_features = []

# Loop para extrair características MFCC
for idx, row in tqdm(df_files.iterrows(), total=len(df_files)):
    file_path = row["file_path"]
    
    try:
        y, sr = librosa.load(file_path, sr=None)
        
        # Extrair MFCCs (13 coeficientes padrão)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        
        # Derivadas delta (1ª e 2ª ordem)
        delta = librosa.feature.delta(mfcc)
        delta2 = librosa.feature.delta(mfcc, order=2)
        
        # Para cada coeficiente MFCC, calcular estatísticas para mfcc, delta e delta2
        feats = {}
        for i in range(13):
            # MFCC
            feats[f"mfcc_{i+1}_mean"] = np.mean(mfcc[i])
            feats[f"mfcc_{i+1}_std"] = np.std(mfcc[i])
            feats[f"mfcc_{i+1}_skew"] = skew(mfcc[i])
            feats[f"mfcc_{i+1}_kurtosis"] = kurtosis(mfcc[i])
            
            # Delta
            feats[f"delta_{i+1}_mean"] = np.mean(delta[i])
            feats[f"delta_{i+1}_std"] = np.std(delta[i])
            feats[f"delta_{i+1}_skew"] = skew(delta[i])
            feats[f"delta_{i+1}_kurtosis"] = kurtosis(delta[i])
            
            # Delta-delta
            feats[f"delta2_{i+1}_mean"] = np.mean(delta2[i])
            feats[f"delta2_{i+1}_std"] = np.std(delta2[i])
            feats[f"delta2_{i+1}_skew"] = skew(delta2[i])
            feats[f"delta2_{i+1}_kurtosis"] = kurtosis(delta2[i])
        
        # Adiciona as informações do participante e segmento
        feats["participant"] = row["participant"]
        feats["segment"] = row["segment"]
        
        mfcc_features.append(feats)
    
    except Exception as e:
        print(f"Erro ao processar {file_path}: {e}")

# Criar DataFrame com as features MFCC
df_mfcc = pd.DataFrame(mfcc_features)

# Salvar para uso futuro
df_mfcc.to_csv("fcg_features_mfcc.csv", index=False)

df_mfcc.head()


100%|██████████| 546/546 [00:27<00:00, 20.00it/s]


,mfcc_1_mean,mfcc_1_std,mfcc_1_skew,mfcc_1_kurtosis,delta_1_mean,delta_1_std,delta_1_skew,delta_1_kurtosis,delta2_1_mean,delta2_1_std,...,delta_13_mean,delta_13_std,delta_13_skew,delta_13_kurtosis,delta2_13_mean,delta2_13_std,delta2_13_skew,delta2_13_kurtosis,participant,segment
0,-561.934204,43.800575,0.230478,-0.765139,-0.887013,8.606462,0.781064,0.459168,1.221687,9.107734,...,0.344950,0.674314,0.480270,-0.419291,0.074981,0.401344,-0.203503,0.156506,Participant_BPF_01,3
1,-580.622070,44.862514,0.919073,0.093048,-1.924359,10.398404,0.141436,-0.461261,1.575409,6.879835,...,0.032461,0.576936,-0.516117,-0.315612,0.085239,0.489153,-0.084693,-1.008502,Participant_BPF_01,4
2,-588.132935,37.640408,0.999098,0.107816,-0.320288,7.428063,-0.212499,-0.786774,2.145930,4.485307,...,-0.133391,0.686617,-0.161648,-0.677261,0.072496,0.711315,0.492462,-0.394351,Participant_BPF_01,5
3,-574.522522,37.008369,1.208569,1.698986,1.925126,5.358781,0.594413,0.318102,2.117136,5.530837,...,0.124098,0.655395,-0.905204,0.750832,0.098660,0.628851,-0.445585,-0.010866,Participant_BPF_01,6
4,-594.598145,27.795744,0.289906,-0.721596,-0.422938,3.157838,-0.816182,0.025133,0.751020,3.615961,...,-0.006917,0.520381,0.151131,-1.239441,0.171621,0.468103,-0.352447,-0.829552,Participant_BPF_01,7
